In [2]:
search_for = 130
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004039764404296875
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 125
all cases: 749112551
type: [1, 1, 1, 1, 130] 130
cases of this type: 285610000
100000 32.26348937138678
200000 66.58458798570182
300000 63.31520754801527
400000 63.83744198772977
500000 71.62489467753498
600000 75.63171957890147
700000 57.26525056855256
800000 67.21714642304593
900000 70.41363315017789
1000000 67.46462956976576
1100000 41.6004692862048
1200000 39.66335997223948
1300000 57.90752512010447
1400000 56.300869983427845
1500000 59.30900376944341
1600000 56.54733923323911
1700000 64.11549687230678
1800000 55.77226247690455
1900000 60.85204378764621
2000000 56.89730800485331
2100000 62.1470780767781
2200000 43.31125411737003
2300000 28.405811698815484
2400000 55.75842585379945
2500000 50.07259219548475
2600000 53.920774571921356
2700000 50.03086718843192
2800000 52.66728167619257
2900000 54.80159727321663
3000000 48.55094798330986

29700000 20.71614089207449
29800000 66.32621546673019
29900000 63.01817871619314
30000000 66.25446826464616
30100000 60.39421339409659
30200000 64.41849023780753
30300000 71.18172947821581
30400000 67.80101013247688
30500000 63.63034541299446
30600000 30.129576729095685
30700000 61.07387010994021
30800000 60.247890888360416
30900000 52.46581612907938
31000000 45.91366238917169
31100000 38.29769095538096
31200000 52.615930459805156
31300000 59.64372563332795
31400000 50.150435377763465
31500000 56.6623054997971
31600000 50.6927963470712
31700000 54.03667367819185
31800000 50.6830267492929
31900000 19.31712867571311
32000000 56.540104775449976
32100000 47.855848632950845
32200000 56.576592555854184
32300000 50.613441620341355
32400000 52.63834829672844
32500000 56.70929704017096
32600000 51.315447379984484
32700000 55.05488853172668
32800000 32.48158288296294
32900000 55.49679800708919
33000000 52.86726071751777
33100000 50.67455414595208
33200000 48.30425000969144
33300000 33.2393056030

59800000 34.41249438118453
59900000 44.25945853592202
60000000 49.022115916010065
60100000 52.70831883472051
60200000 47.94169382547443
60300000 50.268858375433695
60400000 44.19448527334893
60500000 23.722553614768078
60600000 51.60906593375476
60700000 47.63702002002949
60800000 49.51316483862658
60900000 49.42447111330426
61000000 50.67149549864364
61100000 28.9478630356127
61200000 51.165655642776834
61300000 56.36712426438586
61400000 54.3727988841107
61500000 55.450345350567254
61600000 53.36109778503402
61700000 55.949797755225546
61800000 53.518055099859694
61900000 54.509915009233886
62000000 37.94334750985009
62100000 36.66898171240987
62200000 51.87931586590112
62300000 47.35513458187909
62400000 46.53759110713333
62500000 50.471820776418376
62600000 41.754208507719405
62700000 25.134017141141058
62800000 50.01447606760033
62900000 46.77575553472308
63000000 49.664081964873645
63100000 48.7448028796134
63200000 54.45368978965104
63300000 21.84053599028629
63400000 53.5511478

89900000 49.852390173339806
90000000 58.01972263412996
90100000 54.246101315554974
90200000 50.77042111141856
90300000 48.01692015250488
90400000 50.35415895694737
90500000 48.27775423954859
90600000 49.70139441202336
90700000 42.64820948097541
90800000 24.160119256495143
90900000 42.54576305200257
91000000 44.72746465959435
91100000 43.30907869198287
91200000 28.210972118123372
91300000 38.3584034866086
91400000 44.2503096361393
91500000 47.51021944347742
91600000 29.86175689629455
91700000 35.611450478681725
91800000 47.320014047307474
91900000 45.516146417726006
92000000 50.5408304329288
92100000 46.61319567618955
92200000 50.82904981520504
92300000 47.42409482267805
92400000 48.82294357000165
92500000 48.92075589865043
92600000 47.249587347726745
92700000 49.333342938163156
92800000 44.306924139126906
92900000 45.11844077110916
93000000 28.321597283505014
93100000 33.07763356309832
93200000 45.24421382629871
93300000 42.513621301956206
93400000 22.972654449444637
93500000 42.012181

119300000 40.84504106956309
119400000 41.924154033360466
119500000 41.05011932945627
119600000 17.622251821619802
119700000 43.621448544016005
119800000 13.265154960486113
119900000 40.85006371089371
120000000 24.130039466476763
120100000 39.391470399572334
120200000 44.12433101314269
120300000 40.83494104194579
120400000 43.617742085159776
120500000 41.937803539511215
120600000 40.318859712009015
120700000 45.883924315104004
120800000 43.680307242064494
120900000 43.74756210710747
121000000 42.54758084955423
121100000 42.27325801146167
121200000 42.452407965438965
121300000 38.94349121214915
121400000 39.68588165120387
121500000 38.33423428173465
121600000 39.95189145619678
121700000 40.03725096396004
121800000 20.62409086799632
121900000 43.3815320073495
122000000 13.887178455583737
122100000 35.84390218816291
122200000 29.252437614121057
122300000 38.27141474417976
122400000 40.83237181913375
122500000 38.038773133261174
122600000 40.42866348588822
122700000 40.341523803991564
12280

...

639100000 1.5594936788587013
639200000 1.831522953384304
639300000 1.9400018481416834
best so far: 0
type: [1, 1, 2, 13, 5] 130
cases of this type: 5492500
639400000 2.6572041666978006
639500000 1.4968459441907884
639600000 1.9145686875486738
639700000 1.6976338003586855
639800000 1.8617691724238634
639900000 1.3515950552904705
640000000 0.9431847915059149
640100000 0.921906190799417
640200000 1.1988993760633038
640300000 1.2717672222033267
640400000 1.2971437793340068
640500000 1.6274269132481713
640600000 1.8636767154079443
640700000 1.5887582174682437
640800000 1.0563233245700956
640900000 1.2950860260251826
641000000 1.0407491034613907
641100000 1.3894088183193642
641200000 1.2462966250009735
641300000 1.4183259541837117
641400000 0.9612379475002123
641500000 1.2472832734829975
641600000 1.681788638924867
641700000 1.5895860525214136
641800000 1.6790998070081087
641900000 1.657806361351045
642000000 1.8798076297955804
642100000 1.633044380483953
642200000 1.6844407725092947
6423000

665900000 0.4338056867450178
666000000 0.5758034935061633
666100000 0.3752017509918405
666200000 0.4047357951419453
666300000 0.37418113264850245
666400000 0.9830798020107878
666500000 1.5368124840191424
666600000 1.6319164560567159
666700000 1.1964930889554177
666800000 1.763694804301089
666900000 1.5325359504132694
667000000 1.100812594524687
667100000 1.642830439115848
667200000 1.358196908692295
best so far: 0
type: [1, 1, 10, 13, 1] 130
cases of this type: 219700
667300000 0.5984800958910406
667400000 0.31401269126691816
best so far: 0
type: [1, 1, 13, 1, 10] 130
cases of this type: 1690000
667500000 1.0887570738441879
667600000 2.380236147084179
667700000 1.764923449481982
667800000 1.7417490915085894
667900000 1.7640215429294313
668000000 1.2554860947334647
668100000 0.39084805290742514
668200000 0.35373526085597273
668300000 0.47595043081534505
668400000 0.4773094384941332
668500000 0.5789651670375572
668600000 0.5941028647131265
668700000 0.5202116959925778
668800000 0.4909891

691700000 1.5856863873900535
691800000 0.9764646910655955
691900000 2.0351650354486215
692000000 2.3611938814265314
692100000 1.8386365915767273
692200000 1.4462554828104834
692300000 0.775752033420529
692400000 1.163327474365657
692500000 1.7039453969218428
692600000 1.266704776560978
692700000 1.134867731427645
692800000 0.9408011358119952
692900000 0.7929919926132083
693000000 1.3744978390802767
693100000 1.556193058245498
693200000 1.460122956751406
693300000 1.0178626248137455
693400000 1.0671489527524542
693500000 1.314571842761304
693600000 1.3546959001532364
693700000 1.4380883639153197
693800000 1.4218288551209628
693900000 1.2372455132450093
694000000 0.909941860355958
694100000 1.4663658745366177
694200000 1.5991304128047445
694300000 1.7510761018030119
694400000 1.6775807401238163
694500000 0.9558298767426365
694600000 0.9343953278515895
694700000 2.0167610949680626
694800000 2.096759191174866
694900000 1.2552055474185826
695000000 0.995779679742988
695100000 0.908967558083

best so far: 0
type: [1, 2, 13, 1, 5] 130
cases of this type: 211250
718800000 0.19457517166625196
718900000 0.22103898283268875
719000000 0.1819133194936812
best so far: 0
type: [1, 2, 13, 5, 1] 130
cases of this type: 42250
best so far: 0
type: [1, 2, 65, 1, 1] 130
cases of this type: 8450
best so far: 0
type: [1, 5, 1, 1, 26] 130
cases of this type: 2284880
719100000 0.3243998896882951
719200000 0.4357528999847041
719300000 0.3967631954604765
719400000 0.40355208374890417
719500000 0.32510788402978646
719600000 0.32400399212350645
719700000 0.32493982363191315
719800000 0.361595546754086
719900000 0.5171031466718674
720000000 0.5669550858773794
720100000 0.24917235813081265
720200000 0.13373190009068053
720300000 0.13718280872473915
720400000 0.15347480781135098
720500000 0.45783708428598446
720600000 0.4428501687188784
720700000 0.2879138879176464
720800000 0.34871151043265464
720900000 0.42698358554530075
721000000 0.3621490901582049
721100000 0.32624954523026944
721200000 0.37654

739300000 0.33968742212699254
739400000 0.2554721549066603
739500000 0.19583387904775676
739600000 0.18526126919829447
739700000 0.23443637743243576
739800000 0.2931764522523158
739900000 0.2231576833496445
740000000 0.2842943822169443
740100000 0.2174123561860085
740200000 0.29110968789959285
740300000 0.3027586154605945
740400000 0.18469001651826436
740500000 0.1675992551069081
740600000 0.33613858236938127
740700000 0.3064159195322719
740800000 0.25773147841151156
740900000 0.2276408210921347
741000000 0.21398919356654883
741100000 0.19055052614206805
741200000 0.1546335226491511
741300000 0.19243257010865808
741400000 0.18229154688574406
741500000 0.1663303559921682
741600000 0.1739905516492347
741700000 0.2071592058957206
741800000 0.14527151312820316
best so far: 0
type: [2, 1, 1, 5, 13] 130
cases of this type: 3570125
741900000 0.148595371863895
742000000 0.08130211943131288
742100000 0.060988145459644004
742200000 0.06949358865598573
742300000 0.05101231237635745
742400000 0.06